[Ollama RAG Article](https://medium.com/@imabhi1216/implementing-rag-using-langchain-and-ollama-93bdf4a9027c)

# implemeting ollama similarity search expriment

In [24]:
from langchain.document_loaders import PDFPlumberLoader

from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("/Users/tamilselavans/Downloads/udemy", glob="*.pdf", loader_cls=PDFPlumberLoader)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

Number of documents loaded: 26


# example chunks

In [26]:
docs[1].page_content

'INDEX\nS.N Topic Page No.\n1 GENERAL TROUBLESHOOTING 03-09\n2 TROUBLE SHOOTING FOR THREE PHASE VOLTAGE STABILIZERS 10-11\n3 HYDRAULIC SYSTEMS TROUBLE SHOOTING 12-14\n4 PREVENTIVE MAINTENANCE SCHEDULE 15-24\n5 RESOURCES AND SPARES FOR CNC MACHINES 25-26\n2\n'

In [27]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

/var/folders/g4/9z6rn3xs13x2b0bkcf9ljwbw0000gp/T/ipykernel_36638/2762011161.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())


# example chunk with metadata

In [37]:
documents[1]

Document(metadata={'source': '/Users/tamilselavans/Downloads/udemy/cnc_troubleshooting.pdf', 'file_path': '/Users/tamilselavans/Downloads/udemy/cnc_troubleshooting.pdf', 'page': 1, 'total_pages': 26, 'Creator': '', 'Producer': 'https://www.convertapi.com', 'CreationDate': "D:20220227094006+02'00'", 'ModDate': "D:20220227094008+02'00'"}, page_content='INDEX\nS.N Topic Page No. 1 GENERAL TROUBLESHOOTING 03-09\n2 TROUBLE SHOOTING FOR THREE PHASE VOLTAGE STABILIZERS 10-11\n3 HYDRAULIC SYSTEMS TROUBLE SHOOTING 12-14\n4 PREVENTIVE MAINTENANCE SCHEDULE 15-24\n5 RESOURCES AND SPARES FOR CNC MACHINES 25-26\n2\n')

In [38]:
# Check number of chunks created
print("Number of chunks created: ", len(documents))


# Printing first few chunks
for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Number of chunks created:  50

CHUNK : 1
TROUBLESHOOTING GUIDE FOR CNC MACHINES
PATIALA LOCOMOTIVE WORKS, PATIALA
An ISO 9001, ISO 14001, IS 18001, ISO 50001, 5S & Green Building Certified Organization
1


CHUNK : 2
INDEX
S.N Topic Page No. 1 GENERAL TROUBLESHOOTING 03-09
2 TROUBLE SHOOTING FOR THREE PHASE VOLTAGE STABILIZERS 10-11
3 HYDRAULIC SYSTEMS TROUBLE SHOOTING 12-14
4 PREVENTIVE MAINTENANCE SCHEDULE 15-24
5 RESOURCES AND SPARES FOR CNC MACHINES 25-26
2


CHUNK : 3
TROUBLESHOOTING OF CNC MACHINES
Objective
At the end of this unit you should understand:-
• Different type of machine maintenance. • Best maintenance practices followed by CNC machines. • Tools and accessories used in CNC machine tools maintenance work. • Problems related to mechanical system in CNC machines
• Meaning of the term “Backlash”, how to identify and measure Backlash. • Causes of failure of electronic system in CNC machines and precautions to be
observed. • How to check pneumatic system in CNC machines. and

# hf embeding

In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

/var/folders/g4/9z6rn3xs13x2b0bkcf9ljwbw0000gp/T/ipykernel_36638/2495835858.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


# search type  = similarity

In [30]:
# Input
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retrieved_docs = retriever.invoke("MAINTENANCE TOOLS AND ACCESSORIES REQUIRED DURING CNC MACHINE TOOL MAINTENANCE WORK")


# retrieved chunks

In [31]:
retrieved_docs[:4][3].page_content

'TROUBLESHOOTING OF CNC MACHINES\nObjective\nAt the end of this unit you should understand:-\n• Different type of machine maintenance. • Best maintenance practices followed by CNC machines. • Tools and accessories used in CNC machine tools maintenance work. • Problems related to mechanical system in CNC machines\n• Meaning of the term “Backlash”, how to identify and measure Backlash. • Causes of failure of electronic system in CNC machines and precautions to be\nobserved. • How to check pneumatic system in CNC machines. and\n• Causes of excessive noise in CNC Machines and how to eliminate it. DIFFERENT TYPE OF MACHINE MAINTENANCE. 1 CORRECTIVE MAINTENANCE:-\nMaintenance performed, after the occurrence of a failure, to restore the machine\ntool to its original condition, to the extent possible. 2. PREVENTIVE MAINTENANCE:-\nMaintenance preformed to retain the machine in its original condition, to the\nextent possible, by providing systematic inspection, periodical checking\nservicing, de

# Lab machine config for brain llm

In [32]:
# from langchain_community.llms import Ollama

# # Define llm
# llm = Ollama(model="gemma3:27b")

from langchain_community.llms import Ollama

# Use 'base_url' instead of 'api_url' and don't pass max_tokens
llm = Ollama(
    model="gemma3:27b",
    base_url="http://192.168.1.18:11435"  # correct way to override endpoint
)


In [33]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 15-20 sentences with bullet points.
4. end with 1 line answer summary.
Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
                  
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
                  
              )


# Q&A

In [34]:
question = "MAINTENANCE TOOLS AND ACCESSORIES REQUIRED DURING CNC MACHINE TOOL MAINTENANCE WORK"

In [35]:
print(qa(question)["result"])




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 15-20 sentences with bullet points.
4. end with 1 line answer summary.
Context: Context:
content:quick repair and maintenance /trouble shooting. 7. As far as possible carryout reassembly in a clean room. 8. When the electrical cabinet is in open position do not allow the machine in the
running condition. 9. Do not bypass the interlocks. 10. Follow proper procedure and communication protocols when reporting
service/maintenance problems to the service staff of the CNC machine tool
manufacturing company
. MAINTENANCE TOOLS AND ACCESSORIES REQUIRED DURING CNC MACHINE
TOOL MAINTENANCE WORK. 1.
source:/Users/tamilselavans/Downloads/udemy/cnc_troubleshooting.pdf

Context:
content:t